In [ ]:
docker run -dit --name nvidia-10.1-container --gpus all nvidia/cuda:10.1-devel-ubuntu18.04 bash

# Ejemplos

## 1) Programa de hello world

`hello_world.cu`:

In [ ]:
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<3,3>>>(); //3 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


Compilación: `$nvcc hello_world.cu -o hello_world.out`

Ejecución: `$./hello_world.out`:

```
Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hello world del bloque 2 del thread 0!
Hello world del bloque 2 del thread 1!
Hello world del bloque 2 del thread 2!
Hola del cpu thread

```

### Comentarios respecto al programa de `hello_world.cu` anterior

* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

`$nvcc -x cu hello_world.c -o hello_world.out`

* La función `main` se ejecuta en la CPU.

* La función `func` es un *kernel*, lo cual significa que es una función que será ejecutada por *threads* en el *device* (GPU), también llamados *CUDA threads*. Obsérvese que la función `func` inicia con `__global__` para diferenciarla de `main`.

* El llamado a la ejecución del kernel se realiza en el *host* a partir del uso de la sintaxis: `<<< , >>>`. Como se lanzó el programa anterior fueron con $3$ bloques (primera posición), cada uno con $3$ *threads*.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del kernel.

* Los *CUDA threads* son divididos en bloques y todos los bloques se encuentran en un *grid*. En el lanzamiento del *kernel*  se debe especificar al hardware cuántos bloques tendrá nuestro *grid* y cuántos *threads* estarán en cada bloque. Las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los threads. El *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de `blockIdx.x` y `threadIdx.x` refiere a la primera coordenada del bloque y del *thread* en el *grid* y en el bloque respectivamente. 

**Comentario:** la elección del número de bloques en un grid o el número de *threads* en un bloque no corresponde a alguna disposición del hardware, esto es, si se lanza un kernel con `<<< 1, 3 >>>` no implica que la GPU tenga un bloque o 3 *threads*.

## 2) Programa de suma vectorial

`suma_vectorial.cu`:

In [ ]:
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int tid = blockIdx.x;
    if(tid<N) //suposición N menor al número máximo de bloques que se pueden lanzar
    c[tid] = a[tid]+b[tid];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    //alojando en device
    cudaMalloc((void **)&device_a, sizeof(int)*N);
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //copiamos arreglos a, b a la GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    suma_vect<<<N,1>>>(device_a,device_b,device_c); //N bloques de 1 thread
    cudaDeviceSynchronize();
    //copia del resultado al arreglo c:
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

### Comentarios respecto al programa de `suma_vectorial.cu` anterior

* 

### Regla compuesta del rectángulo

In [ ]:
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [ ]:
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
double x=0.0;

if(threadIdx.x<=n-1){
x=a+(threadIdx.x+1/2.0)*h_hat;
data[threadIdx.x]=std::exp(-std::pow(x,2));
}
    *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
}


int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    return 0;
}

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01

Error relativo de la solución: 4.104931878976858e-08

**Referencias**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. [CUDA](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA)